<a href="https://colab.research.google.com/github/ymkge/competiton/blob/main/signate%E3%82%B3%E3%83%B3%E3%83%9A_%E6%A0%AA%E4%BE%A1%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=9ba14c16a665aa2c483b796dfa2a37d1644a4b8f6892a9d872232444b8b3799e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


## google driveへのアクセス(実行前に signateで発行したapikeyをgoogle driveへ格納しておく)

In [2]:
auth.authenticate_user()

In [3]:
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

In [4]:
filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


## コンペの一覧を表示

In [5]:
!signate list

  competitionId  title                                                                              closing     prize              submitters
---------------  ---------------------------------------------------------------------------------  ----------  ---------------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                               -                                    7206
             24  【練習問題】お弁当の需要予測                                                       -                                    8803
             27  【練習問題】Jリーグの観客動員数予測                                                -                                    2167
            100  【練習問題】手書き文字認識                                                         -           Knowledge                 351
            102  【練習問題】タイタニックの生存予測                                                 -           Knowledge                3077
            103  【練習問題】音楽ラベリング                                                       

## コンペ用データの取得

In [6]:
!signate files --competition-id=409

  fileId  name                   title                     size  updated_at
--------  ---------------------  ----------------------  ------  -------------------
    1598  train.csv              学習用データ             74502  2020-12-08 22:18:22
    1599  test.csv               評価用データ             46220  2020-12-08 22:18:35
    1600  sample_submission.csv  応募用サンプルファイル    2600  2020-12-08 22:18:52


In [7]:
!signate download --competition-id=409

sample_submission.csv

test.csv

train.csv


Download completed.


In [8]:
!ls -l

total 132
drwxr-xr-x 1 root root  4096 Sep 25 18:24 sample_data
-rw-r--r-- 1 root root  2600 Sep 29 12:35 sample_submission.csv
-rw-r--r-- 1 root root 46220 Sep 29 12:35 test.csv
-rw-r--r-- 1 root root 74502 Sep 29 12:35 train.csv


In [89]:
import pandas as pd

df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_sample_submission = pd.read_csv('sample_submission.csv')

In [90]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2026 non-null   object 
 1   Open    2026 non-null   float64
 2   High    2026 non-null   float64
 3   Low     2026 non-null   float64
 4   Close   2026 non-null   float64
 5   Up      2026 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 95.1+ KB


In [91]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1200 non-null   object 
 1   Open    1200 non-null   float64
 2   High    1200 non-null   float64
 3   Low     1200 non-null   float64
 4   Close   1200 non-null   float64
dtypes: float64(4), object(1)
memory usage: 47.0+ KB


In [92]:
df_sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   2012-01-26  199 non-null    object
 1   0           199 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [93]:
df_train

,Date,Open,High,Low,Close,Up
0,2004-01-02,50.85,50.91,50.62,50.72,1
1,2004-01-05,50.75,50.84,50.61,50.79,1
2,2004-01-06,50.87,51.26,50.86,51.15,1
3,2004-01-07,51.45,51.54,51.29,51.51,0
4,2004-01-08,51.15,51.33,51.02,51.28,0
...,...,...,...,...,...,...
2021,2012-01-11,108.62,109.00,108.41,108.86,1
2022,2012-01-12,109.68,109.90,109.44,109.83,0
2023,2012-01-13,109.53,109.72,109.08,109.66,1
2024,2012-01-17,109.99,110.47,109.57,109.96,1


In [94]:
df_test

,Date,Open,High,Low,Close
0,2012-01-19,110.79,110.82,109.96,110.39
1,2012-01-20,110.45,111.10,110.36,110.73
2,2012-01-23,110.95,111.10,109.65,110.36
3,2012-01-24,110.16,110.16,108.13,108.39
4,2012-01-25,106.25,107.11,105.97,106.89
...,...,...,...,...,...
1195,2019-12-17,185.52,186.40,184.62,184.68
1196,2019-12-18,183.86,184.01,182.38,183.10
1197,2019-12-19,183.83,185.17,183.66,185.08
1198,2019-12-20,186.29,186.64,184.81,185.19


In [95]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2026 non-null   object 
 1   Open    2026 non-null   float64
 2   High    2026 non-null   float64
 3   Low     2026 non-null   float64
 4   Close   2026 non-null   float64
 5   Up      2026 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 95.1+ KB


- モデル作成時に、日時情報も考慮した形に作成しなおす
- その上でdf_testのデータを使って予測結果を出力して、signateに提出＆すこ

In [17]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [63]:
import pandas as pd
import yfinance as yf
from pycaret.regression import *

In [64]:
df_train

,Date,Open,High,Low,Close,Up
0,2004-01-02,50.85,50.91,50.62,50.72,1
1,2004-01-05,50.75,50.84,50.61,50.79,1
2,2004-01-06,50.87,51.26,50.86,51.15,1
3,2004-01-07,51.45,51.54,51.29,51.51,0
4,2004-01-08,51.15,51.33,51.02,51.28,0
...,...,...,...,...,...,...
2021,2012-01-11,108.62,109.00,108.41,108.86,1
2022,2012-01-12,109.68,109.90,109.44,109.83,0
2023,2012-01-13,109.53,109.72,109.08,109.66,1
2024,2012-01-17,109.99,110.47,109.57,109.96,1


In [96]:
# 特徴量エンジニアリング
df_train['MA_7'] = df_train['Close'].rolling(window=7).mean()
df_train['MA_30'] = df_train['Close'].rolling(window=30).mean()
df_train['MA_90'] = df_train['Close'].rolling(window=90).mean()
df_train['Return'] = df_train['Close'].pct_change()

# NaNを含む行を削除
df_train = df_train.dropna()
df_train

,Date,Open,High,Low,Close,Up,MA_7,MA_30,MA_90,Return
89,2004-05-11,52.12,52.21,51.85,52.02,0,52.485714,52.966667,52.627222,0.001733
90,2004-05-12,51.76,52.20,51.32,51.61,1,52.322857,52.895333,52.637111,-0.007882
91,2004-05-13,51.77,51.86,51.51,51.73,1,52.151429,52.822000,52.647556,0.002325
92,2004-05-14,51.74,52.04,51.59,51.80,0,51.997143,52.751000,52.654778,0.001353
93,2004-05-17,51.54,51.77,51.34,51.63,0,51.848571,52.673333,52.656111,-0.003282
...,...,...,...,...,...,...,...,...,...,...
2021,2012-01-11,108.62,109.00,108.41,108.86,1,108.891429,107.933333,103.860556,-0.002383
2022,2012-01-12,109.68,109.90,109.44,109.83,0,109.095714,108.118000,103.960111,0.008911
2023,2012-01-13,109.53,109.72,109.08,109.66,1,109.448571,108.245667,104.062778,-0.001548
2024,2012-01-17,109.99,110.47,109.57,109.96,1,109.548571,108.381000,104.150556,0.002736


In [97]:
# 目的変数と説明変数の設定
target = 'Up'
features = ['Open', 'High', 'Low', 'Close', 'MA_7', 'MA_30', 'MA_90', 'Return']

In [98]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1937 entries, 89 to 2025
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1937 non-null   object 
 1   Open    1937 non-null   float64
 2   High    1937 non-null   float64
 3   Low     1937 non-null   float64
 4   Close   1937 non-null   float64
 5   Up      1937 non-null   int64  
 6   MA_7    1937 non-null   float64
 7   MA_30   1937 non-null   float64
 8   MA_90   1937 non-null   float64
 9   Return  1937 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 231.0+ KB


In [68]:
# PyCaretのセットアップ
s = setup(data=df_train, target=target, train_size=0.8, session_id=123,
          numeric_features=features, use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Description,Value
0,Session id,123
1,Target,Up
2,Target type,Regression
3,Original data shape,"(1937, 10)"
4,Transformed data shape,"(1937, 10)"
5,Transformed train set shape,"(1549, 10)"
6,Transformed test set shape,"(388, 10)"
7,Numeric features,8
8,Categorical features,1
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

In [69]:
# 最適なモデルの選択
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,0.4922,0.2430,0.4929,0.0231,0.3469,0.4793,0.0960
llar,Lasso Least Angle Regression,0.4999,0.2501,0.5001,-0.0054,0.3516,0.4868,0.0900
lr,Linear Regression,0.4999,0.2501,0.5001,-0.0054,0.3516,0.4868,0.0970
br,Bayesian Ridge,0.4999,0.2501,0.5001,-0.0054,0.3516,0.4868,0.1760
dummy,Dummy Regressor,0.4999,0.2501,0.5001,-0.0054,0.3516,0.4868,0.0650
en,Elastic Net,0.4999,0.2501,0.5001,-0.0054,0.3516,0.4868,0.0940
lasso,Lasso Regression,0.4999,0.2501,0.5001,-0.0054,0.3516,0.4868,0.0910
omp,Orthogonal Matching Pursuit,0.5003,0.2506,0.5006,-0.0074,0.3519,0.4872,0.1530
et,Extra Trees Regressor,0.4993,0.2507,0.5007,-0.0077,0.3545,0.4630,0.2560
lar,Least Angle Regression,0.4949,0.2614,0.5107,-0.0507,0.3564,0.4826,0.0930


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [70]:
# モデルのチューニング
tuned_model = tune_model(best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4955,0.2527,0.5027,-0.0107,0.3552,0.4762
1,0.4686,0.2266,0.4760,0.0934,0.3372,0.4487
2,0.4672,0.2257,0.4750,0.0836,0.3286,0.4615
3,0.4709,0.2279,0.4774,0.0796,0.3315,0.4669
4,0.4776,0.2366,0.4864,0.0446,0.3498,0.4515
5,0.4707,0.2322,0.4819,0.0709,0.3395,0.4618
6,0.4677,0.2267,0.4761,0.0914,0.3410,0.4409
7,0.4576,0.2161,0.4649,0.1251,0.3219,0.4562
8,0.4728,0.2280,0.4775,0.0878,0.3366,0.4702


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [71]:
# 最終モデルの作成
final_model = finalize_model(tuned_model)

In [99]:
# 特徴量エンジニアリング(テスト用データ)
df_test['MA_7'] = df_test['Close'].rolling(window=7).mean()
df_test['MA_30'] = df_test['Close'].rolling(window=30).mean()
df_test['MA_90'] = df_test['Close'].rolling(window=90).mean()
df_test['Return'] = df_test['Close'].pct_change()

# NaNを含む行を削除
# df_test = df_test.dropna()
df_test

,Date,Open,High,Low,Close,MA_7,MA_30,MA_90,Return
0,2012-01-19,110.79,110.82,109.96,110.39,NaN,NaN,NaN,NaN
1,2012-01-20,110.45,111.10,110.36,110.73,NaN,NaN,NaN,0.003080
2,2012-01-23,110.95,111.10,109.65,110.36,NaN,NaN,NaN,-0.003341
3,2012-01-24,110.16,110.16,108.13,108.39,NaN,NaN,NaN,-0.017851
4,2012-01-25,106.25,107.11,105.97,106.89,NaN,NaN,NaN,-0.013839
...,...,...,...,...,...,...,...,...,...
1195,2019-12-17,185.52,186.40,184.62,184.68,183.680000,185.369000,192.005000,-0.010183
1196,2019-12-18,183.86,184.01,182.38,183.10,183.834286,184.936667,191.957222,-0.008555
1197,2019-12-19,183.83,185.17,183.66,185.08,184.278571,184.523000,191.938222,0.010814
1198,2019-12-20,186.29,186.64,184.81,185.19,184.608571,184.142667,191.919333,0.000594


In [100]:
# テストデータでの予測
predictions = predict_model(final_model, data=df_test)

# 予測結果が0.5以上なら1を設定する。それ以外なら０を設定
predictions['Up'] = (predictions['prediction_label'] >= 0.5).astype(int)

In [101]:
# 結果の表示
print(predictions[['Date', 'Close', 'prediction_label', 'Up']])

            Date       Close  prediction_label  Up
0     2012-01-19  110.389999         -2.680434   0
1     2012-01-20  110.730003         -2.571902   0
2     2012-01-23  110.360001         -2.720621   0
3     2012-01-24  108.389999         -2.733538   0
4     2012-01-25  106.889999         -2.211258   0
...          ...         ...               ...  ..
1195  2019-12-17  184.679993          0.483234   0
1196  2019-12-18  183.100006          0.585011   1
1197  2019-12-19  185.080002          0.794884   1
1198  2019-12-20  185.190002          0.444549   0
1199  2019-12-23  183.339996          0.608315   1

[1200 rows x 4 columns]


In [102]:
df_sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   2012-01-26  199 non-null    object
 1   0           199 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [103]:
# 列名を確認した上で、適切な列名を使用してデータフレームを変換
df_transformed = pd.DataFrame({
    'Date': [df_sample_submission.columns[0]] + df_sample_submission.iloc[:, 0].tolist(),
    'Up': [df_sample_submission.iloc[0, 1]] + df_sample_submission.iloc[:, 1].tolist()
})

# インデックスをリセット
df_transformed.index = range(len(df_transformed))

# 日付をdatetime型に変換
df_transformed['Date'] = pd.to_datetime(df_transformed['Date'])

# 結果の表示
print(df_transformed)

          Date  Up
0   2012-01-26   0
1   2012-02-09   0
2   2012-02-24   0
3   2012-03-09   0
4   2012-03-23   0
..         ...  ..
195 2019-10-25   0
196 2019-11-08   0
197 2019-11-22   0
198 2019-12-09   0
199 2019-12-23   0

[200 rows x 2 columns]


In [104]:
df_transformed

,Date,Up
0,2012-01-26,0
1,2012-02-09,0
2,2012-02-24,0
3,2012-03-09,0
4,2012-03-23,0
...,...,...
195,2019-10-25,0
196,2019-11-08,0
197,2019-11-22,0
198,2019-12-09,0


In [105]:
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    200 non-null    datetime64[ns]
 1   Up      200 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.2 KB


In [106]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Date              1200 non-null   category
 1   Open              1200 non-null   float32 
 2   High              1200 non-null   float32 
 3   Low               1200 non-null   float32 
 4   Close             1200 non-null   float32 
 5   MA_7              1194 non-null   float32 
 6   MA_30             1171 non-null   float32 
 7   MA_90             1111 non-null   float32 
 8   Return            1199 non-null   float32 
 9   prediction_label  1200 non-null   float64 
 10  Up                1200 non-null   int64   
dtypes: category(1), float32(8), float64(1), int64(1)
memory usage: 100.4 KB


In [107]:
# category型からdatetime64型への変換
predictions['Date'] = pd.to_datetime(predictions['Date'].astype(str))

In [108]:
predictions

,Date,Open,High,Low,Close,MA_7,MA_30,MA_90,Return,prediction_label,Up
0,2012-01-19,110.790001,110.820000,109.959999,110.389999,NaN,NaN,NaN,NaN,-2.680434,0
1,2012-01-20,110.449997,111.099998,110.360001,110.730003,NaN,NaN,NaN,0.003080,-2.571902,0
2,2012-01-23,110.949997,111.099998,109.650002,110.360001,NaN,NaN,NaN,-0.003341,-2.720621,0
3,2012-01-24,110.160004,110.160004,108.129997,108.389999,NaN,NaN,NaN,-0.017851,-2.733538,0
4,2012-01-25,106.250000,107.110001,105.970001,106.889999,NaN,NaN,NaN,-0.013839,-2.211258,0
...,...,...,...,...,...,...,...,...,...,...,...
1195,2019-12-17,185.520004,186.399994,184.619995,184.679993,183.679993,185.369003,192.005005,-0.010183,0.483234,0
1196,2019-12-18,183.860001,184.009995,182.380005,183.100006,183.834290,184.936661,191.957230,-0.008555,0.585011,1
1197,2019-12-19,183.830002,185.169998,183.660004,185.080002,184.278564,184.522995,191.938217,0.010814,0.794884,1
1198,2019-12-20,186.289993,186.639999,184.809998,185.190002,184.608566,184.142670,191.919327,0.000594,0.444549,0


In [109]:
df_transformed

,Date,Up
0,2012-01-26,0
1,2012-02-09,0
2,2012-02-24,0
3,2012-03-09,0
4,2012-03-23,0
...,...,...
195,2019-10-25,0
196,2019-11-08,0
197,2019-11-22,0
198,2019-12-09,0


In [110]:
df_train

,Date,Open,High,Low,Close,Up,MA_7,MA_30,MA_90,Return
89,2004-05-11,52.12,52.21,51.85,52.02,0,52.485714,52.966667,52.627222,0.001733
90,2004-05-12,51.76,52.20,51.32,51.61,1,52.322857,52.895333,52.637111,-0.007882
91,2004-05-13,51.77,51.86,51.51,51.73,1,52.151429,52.822000,52.647556,0.002325
92,2004-05-14,51.74,52.04,51.59,51.80,0,51.997143,52.751000,52.654778,0.001353
93,2004-05-17,51.54,51.77,51.34,51.63,0,51.848571,52.673333,52.656111,-0.003282
...,...,...,...,...,...,...,...,...,...,...
2021,2012-01-11,108.62,109.00,108.41,108.86,1,108.891429,107.933333,103.860556,-0.002383
2022,2012-01-12,109.68,109.90,109.44,109.83,0,109.095714,108.118000,103.960111,0.008911
2023,2012-01-13,109.53,109.72,109.08,109.66,1,109.448571,108.245667,104.062778,-0.001548
2024,2012-01-17,109.99,110.47,109.57,109.96,1,109.548571,108.381000,104.150556,0.002736


In [111]:
df_test.head()

,Date,Open,High,Low,Close,MA_7,MA_30,MA_90,Return
0,2012-01-19,110.79,110.82,109.96,110.39,NaN,NaN,NaN,NaN
1,2012-01-20,110.45,111.10,110.36,110.73,NaN,NaN,NaN,0.003080
2,2012-01-23,110.95,111.10,109.65,110.36,NaN,NaN,NaN,-0.003341
3,2012-01-24,110.16,110.16,108.13,108.39,NaN,NaN,NaN,-0.017851
4,2012-01-25,106.25,107.11,105.97,106.89,NaN,NaN,NaN,-0.013839


In [115]:
# df1とdf2をdate列でマージ
df_merged = pd.merge(df_transformed['Date'], predictions[['Date', 'Up']], on='Date', how='left')

# 結果の表示
print("\nマージ後のdf1:")
print(df_merged)


マージ後のdf1:
          Date  Up
0   2012-01-26   0
1   2012-02-09   0
2   2012-02-24   0
3   2012-03-09   1
4   2012-03-23   0
..         ...  ..
195 2019-10-25   1
196 2019-11-08   0
197 2019-11-22   1
198 2019-12-09   0
199 2019-12-23   1

[200 rows x 2 columns]


In [116]:
# 予測結果をDataFrameに格納
# predictions_df = pd.DataFrame({'id': test_ids, 'prediction': predictions})

# CSVファイルとして保存
df_merged.to_csv('predictions.csv', index=False)